In [1]:
import geopandas
import pandas as pd
import numpy as np
import random
import math
import seaborn as sns
import matplotlib.pyplot as plt

# Data

In [2]:
buildings = geopandas.read_file('/home/ai6644/Malmo/Data/Sjöbo/GIS/FastighetskartanBebyggelseVektor_1801_3006/by_get.shp')
buildings = buildings.to_crs("EPSG:4326")
buildings.geometry = buildings.geometry.centroid

/tmp/ipykernel_7614/2813531054.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buildings.geometry = buildings.geometry.centroid


In [2]:
ruta1000 = geopandas.read_file('/home/ai6644/Malmo/Data/Sjöbo/GIS/B1 ruta 1000 difference with 250.shp')
ruta1000 = ruta1000.to_crs("EPSG:4326")

ruta250 = geopandas.read_file('/home/ai6644/Malmo/Data/Sjöbo/GIS/B1 ruta 250.shp')
ruta250 = ruta250.to_crs("EPSG:4326")

rutas = geopandas.GeoDataFrame(pd.concat([ruta250, ruta1000], axis=0))

rutas['Ruta'] = rutas.Ruta.astype(int)
rutas = rutas.set_index('Ruta')

In [4]:
houses = buildings[buildings.ANDAMAL_1 < 200]
workplaces = geopandas.read_file('data/workplaces_OSM_SCB_merged.geojson')

In [3]:
rutas

,Rutstorl,Ald0_6,Ald7_15,Ald16_19,Ald20_24,Ald25_44,Ald45_64,Ald65_W,Totalt,geometry
Ruta,,,,,,,,,,
3495006225750,250,4,14,7,0,13,19,9,66,"POLYGON ((12.57704 56.15298, 12.57690 56.15523..."
3490006226500,250,13,29,16,4,29,50,33,174,"POLYGON ((12.56858 56.15956, 12.56843 56.16180..."
3490006226750,250,0,0,0,0,5,5,10,20,"POLYGON ((12.56843 56.16180, 12.56829 56.16405..."
3495006224750,250,3,10,7,3,11,36,30,100,"POLYGON ((12.57761 56.14401, 12.57747 56.14625..."
3500006224250,250,0,3,0,0,4,3,5,15,"POLYGON ((12.58593 56.13968, 12.58579 56.14192..."
...,...,...,...,...,...,...,...,...,...,...
4310006169000,1000,4,4,0,3,13,6,10,40,"POLYGON ((13.90319 55.66193, 13.90294 55.67092..."
4310006196000,1000,0,3,0,0,3,4,0,10,"POLYGON ((13.89636 55.90449, 13.89610 55.91347..."
4320006144000,1000,0,0,0,0,0,3,6,9,"POLYGON ((13.92523 55.43747, 13.92499 55.44646..."


In [77]:
houses_in_ruta = []
workplaces_in_ruta = []
columns = rutas.reset_index().columns
r = rutas.sample(100)
for ruta in r.itertuples():
    ruta_repeated = geopandas.GeoDataFrame(np.repeat([ruta], len(houses.index), axis=0), columns=columns, geometry='geometry', crs="EPSG:4326")
    ruta_houses_ind = houses[ruta_repeated.contains(houses.geometry, align=False).values].index
    houses_in_ruta.append(ruta_houses_ind.to_list())
    
    ruta_repeated = geopandas.GeoDataFrame(np.repeat([ruta], len(workplaces.index), axis=0), columns=columns, geometry='geometry', crs="EPSG:4326")
    ruta_workplaces_ind = workplaces[ruta_repeated.contains(workplaces.geometry, align=False).values].index
    workplaces_in_ruta.append(ruta_workplaces_ind.to_list())

/home/ai6644/miniconda3/envs/gis/lib/python3.9/site-packages/numpy/core/fromnumeric.py:43: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result = getattr(asarray(obj), method)(*args, **kwds)
/home/ai6644/miniconda3/envs/gis/lib/python3.9/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)
/home/ai6644/miniconda3/envs/gis/lib/python3.9/site-packages/numpy/core/fromnumeric.py:43: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the 

In [78]:
r['houses_in_ruta'] = houses_in_ruta
r['workplaces_in_ruta'] = workplaces_in_ruta

In [79]:
r

,Rutstorl,Ald0_6,Ald7_15,Ald16_19,Ald20_24,Ald25_44,Ald45_64,Ald65_W,Totalt,geometry,houses_in_ruta,workplaces_in_ruta
Ruta,,,,,,,,,,,,
4080006144000,1000,0,0,0,0,0,4,0,4,"POLYGON ((13.54602 55.43355, 13.54569 55.44254...","[73432, 75461, 151811]",[]
4280006204000,1000,0,0,0,0,0,0,4,4,"POLYGON ((13.84625 55.97592, 13.84598 55.98490...","[26385, 218097]",[]
3680006231000,1000,0,0,4,0,3,6,3,16,"POLYGON ((12.87203 56.20560, 12.87153 56.21458...","[119525, 153995, 154053, 259587]",[1919]
3800006180000,1000,4,6,0,0,9,10,7,36,"POLYGON ((13.08808 55.75080, 13.08764 55.75978...","[93910, 171688, 186880, 186966, 187223, 188055...",[]
4340006214000,1000,3,3,0,0,5,0,7,18,"POLYGON ((13.93992 56.06661, 13.93967 56.07560...","[128286, 140981, 155358, 197248, 198263, 25468...","[1659, 5534]"
...,...,...,...,...,...,...,...,...,...,...,...,...
3642506195750,250,18,21,3,7,39,59,45,192,"POLYGON ((12.82947 55.88806, 12.82935 55.89030...","[104730, 196193, 200329, 281396, 281524, 28160...",[]
3850006174500,250,29,40,12,46,127,112,69,435,"POLYGON ((13.17001 55.70262, 13.16991 55.70486...","[14683, 14840, 16077, 16675, 16876, 16877, 168...","[19250, 24646, 30904]"
4305006167500,250,0,0,0,0,0,3,0,3,"POLYGON ((13.89562 55.64839, 13.89556 55.65063...",[62955],[]


In [80]:
r.drop('geometry', axis=1).to_feather('data/rutas_with_buildings.feather')

/tmp/ipykernel_7614/4042759684.py:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  r.drop('geometry', axis=1).to_feather('data/rutas_with_buildings.feather')


In [81]:
pd.read_feather('data/rutas_with_buildings.feather')

,Rutstorl,Ald0_6,Ald7_15,Ald16_19,Ald20_24,Ald25_44,Ald45_64,Ald65_W,Totalt,houses_in_ruta,workplaces_in_ruta
Ruta,,,,,,,,,,,
4080006144000,1000,0,0,0,0,0,4,0,4,"[73432, 75461, 151811]",[]
4280006204000,1000,0,0,0,0,0,0,4,4,"[26385, 218097]",[]
3680006231000,1000,0,0,4,0,3,6,3,16,"[119525, 153995, 154053, 259587]",[1919]
3800006180000,1000,4,6,0,0,9,10,7,36,"[93910, 171688, 186880, 186966, 187223, 188055...",[]
4340006214000,1000,3,3,0,0,5,0,7,18,"[128286, 140981, 155358, 197248, 198263, 25468...","[1659, 5534]"
...,...,...,...,...,...,...,...,...,...,...,...
3642506195750,250,18,21,3,7,39,59,45,192,"[104730, 196193, 200329, 281396, 281524, 28160...",[]
3850006174500,250,29,40,12,46,127,112,69,435,"[14683, 14840, 16077, 16675, 16876, 16877, 168...","[19250, 24646, 30904]"
4305006167500,250,0,0,0,0,0,3,0,3,[62955],[]


# Other types

In [5]:
healthcare = geopandas.read_file('data/healthcare.geojson')
school = geopandas.read_file('data/schools.geojson')
pickup_kids = geopandas.read_file('data/pickup_kids.geojson')
shopping_livsmedel = geopandas.read_file('data/shopping_livsmedel.geojson')
shopping_other = geopandas.read_file('data/shopping_other.geojson')
bank_post_mndighet = geopandas.read_file('data/bank_post_mndighet.geojson')
sport_outdoor = geopandas.read_file('data/sport_outdoor.geojson')
association_activity = geopandas.read_file('data/association_activity.geojson')
leisure = geopandas.read_file('data/leisure.geojson')
waste = geopandas.read_file('data/waste.geojson')
goods = geopandas.read_file('data/goods.geojson')

In [40]:
healthcare_in_ruta = []
school_in_ruta = []
pickup_kids_in_ruta = []
shopping_livsmedel_in_ruta = []
shopping_other_in_ruta = []
bank_post_mndighet_in_ruta = []
sport_outdoor_in_ruta = []
association_activity_in_ruta = []
leisure_in_ruta = []
waste_in_ruta = []
goods_in_ruta = []

arrays = (healthcare_in_ruta, school_in_ruta, pickup_kids_in_ruta, shopping_livsmedel_in_ruta,
shopping_other_in_ruta, bank_post_mndighet_in_ruta, sport_outdoor_in_ruta, association_activity_in_ruta,
leisure_in_ruta, waste_in_ruta, goods_in_ruta)

sources = (healthcare, school, pickup_kids, shopping_livsmedel ,shopping_other,
bank_post_mndighet, sport_outdoor , association_activity, leisure, waste, goods)

columns = rutas.reset_index().columns
r = rutas.sample(100)
for ruta in r.itertuples():
    for source, array in zip(sources, arrays):
        ruta_repeated = geopandas.GeoDataFrame(np.repeat([ruta], len(source.index), axis=0), columns=columns, geometry='geometry', crs="EPSG:4326")
        ruta_source_ind = source[ruta_repeated.contains(source.geometry, align=False).values].index
        array.append(ruta_source_ind.to_list())


/home/ai6644/miniconda3/envs/gis/lib/python3.9/site-packages/numpy/core/fromnumeric.py:43: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result = getattr(asarray(obj), method)(*args, **kwds)
/home/ai6644/miniconda3/envs/gis/lib/python3.9/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)
/home/ai6644/miniconda3/envs/gis/lib/python3.9/site-packages/numpy/core/fromnumeric.py:43: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the 

/home/ai6644/miniconda3/envs/gis/lib/python3.9/site-packages/numpy/core/fromnumeric.py:43: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result = getattr(asarray(obj), method)(*args, **kwds)
/home/ai6644/miniconda3/envs/gis/lib/python3.9/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)
/home/ai6644/miniconda3/envs/gis/lib/python3.9/site-packages/numpy/core/fromnumeric.py:43: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the 

In [41]:
names = ('healthcare_in_ruta', 'school_in_ruta', 'pickup_kids_in_ruta', 'shopping_livsmedel_in_ruta',
'shopping_other_in_ruta', 'bank_post_mndighet_in_ruta', 'sport_outdoor_in_ruta', 'association_activity_in_ruta',
'leisure_in_ruta', 'waste_in_ruta', 'goods_in_ruta')

for name, array in zip(names, arrays):
    r[name] = array

In [42]:
r.drop(['geometry', 'Ald0_6','Ald7_15','Ald16_19','Ald20_24','Ald25_44','Ald45_64','Ald65_W','Totalt'], axis=1).to_feather('data/rutas_with_other_buildings.feather')

/tmp/ipykernel_14978/430121545.py:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  r.drop(['geometry', 'Ald0_6','Ald7_15','Ald16_19','Ald20_24','Ald25_44','Ald45_64','Ald65_W','Totalt'], axis=1).to_feather('data/rutas_with_other_buildings.feather')
